In [ ]:
from pyspark import SparkContext
from scipy import sparse as sm
from sklearn.preprocessing import normalize
import numpy as np
import csv
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import pearsonr as pears
from collections import defaultdict
sc = SparkContext.getOrCreate()

In [ ]:
train_rdd = sc.textFile("data/train.csv")
icm_rdd = sc.textFile("data/icm_fede.csv")
test_rdd= sc.textFile("data/target_users.csv")

train_header = train_rdd.first()
icm_header = icm_rdd.first()
test_header= test_rdd.first()

train_clean_data = train_rdd.filter(lambda x: x != train_header).map(lambda line: line.split(',')).map(lambda x: (int(x[0]), int(x[1]), float(x[2])))
icm_clean_data = icm_rdd.filter(lambda x: x != icm_header).map(lambda line: line.split(',')).map(lambda x: (int(x[0]), int(x[1])))
test_clean_data= test_rdd.filter(lambda x: x != test_header).map(lambda line: line.split(','))

test_users=test_clean_data.map( lambda x: int(x[0])).collect()


grouped_rates = train_clean_data.filter(lambda x: x[0] in test_users).map(lambda x: (x[0],x[1])).groupByKey().map(lambda x: (x[0], list(x[1]))).collect()
grouped_rates_dic = dict(grouped_rates)


item_ratings = train_clean_data.map(lambda x: (x[0], x[2])).aggregateByKey((0,0), lambda x,y: (x[0] + y, x[1] + 1),lambda x,y: (x[0] + y[0], x[1] + y[1]))
user_ratings_mean = item_ratings.mapValues(lambda x: (x[0] / (x[1]))).collect()
user_ratings_mean_dic=dict(user_ratings_mean)


item_ratings_forTop = train_clean_data.map(lambda x: (x[1], x[2])).aggregateByKey((0,0), lambda x,y: (x[0] + y, x[1] + 1),lambda x,y: (x[0] + y[0], x[1] + y[1]))#.sortBy(lambda x: x[1][1], ascending=False)
#item_ratings.take(10)
shrinkage_factor = 5
item_ratings_mean = item_ratings_forTop.mapValues(lambda x: (x[0] / (x[1] + shrinkage_factor))).sortBy(lambda x: x[1], ascending = False).map(lambda x: x[0]).collect()


users = train_clean_data.map(lambda x: x[0]).collect()
items = train_clean_data.map(lambda x: x[1]).collect()
ratings = train_clean_data.map(lambda x: x[2]).collect()
ratings_unbiased = train_clean_data.map(lambda x: x[2]-user_ratings_mean_dic[x[0]]).collect()

items_for_features= icm_clean_data.map(lambda x:x[0]).collect()
features = icm_clean_data.map(lambda x:x[1]).collect()
items_for_features.append(37142)
features.append(0)


unos=[1]*len(items_for_features)

UxI= sm.csr_matrix((ratings, (users, items)))
UxI_unbiased= sm.csr_matrix((ratings_unbiased, (users, items)))
IxF= sm.csr_matrix((unos, (items_for_features, features)))

In [ ]:
'''begin of content based'''
IxF_normalized=normalize(IxF,axis=1)
NumItems,NumFeatures=IxF.shape
NumFeatures
IDF=[0]*NumFeatures
for i in range(NumFeatures):
    IDF[i]=np.log10(NumItems/len(IxF.getcol(i).nonzero()[1]))
UxF=UxI.dot(IxF_normalized)
FxI=IxF_normalized.multiply(IDF).T
UxI_pred_CB=UxF.dot(FxI)

In [ ]:
'''contet versione prof'''
IxF_idf=IxF.multiply(IDF)
IxI_sim=sm.csr_matrix(cosine_similarity(IxF_idf))
IxI_sim.setdiag(0)
UxI_pred_CBS=UxI.dot(normalize(IxI_sim,axis=0,norm='l1' ))

In [ ]:
'''collaborative user based'''
UxU_sim_dafile=sc.textFile("users-users-sims.csv").map(lambda x: x.replace("(","").replace(")","").replace(" ","").split(",")).map(lambda x: (int(x[0]), int(x[1]), float(x[2])))
us1=UxU_sim_dafile.map(lambda x:x[0]).collect()
us2=UxU_sim_dafile.map(lambda x:x[1]).collect()
simsus=UxU_sim_dafile.map(lambda x:x[2]).collect()
UxU_sim= sm.csr_matrix((simsus, (us1, us2)))
UxU_sim.setdiag(0)


In [ ]:
k=75
nz_us_row,nz_us_col=UxU_sim.nonzero()
actual_row=-1
actual_top=[]
for i in range(len(nz_us_row)):
    us_row=nz_us_row[i]
    us_col=nz_us_col[i]
    if us_row!=actual_row:
        actual_row=us_row
        actual_top=UxU_sim.getrow(us_row).toarray().argpartition(-k)[-k:]
    if us_col not in actual_top:
        UxU_sim[us_row,us_col]=0
    print(i/len(nz_us_row))

In [ ]:
UxI_pred_CFU=UxU_sim.dot(UxI_unbiased)

In [ ]:
'''collaborative item based'''
IxI_sim_dafile=sc.textFile("items-items-sims.csv").map(lambda x: x.replace("(","").replace(")","").replace(" ","").split(",")).map(lambda x: (int(x[0]), int(x[1]), float(x[2])))
it1=IxI_sim_dafile.map(lambda x:x[0]).collect()
it2=IxI_sim_dafile.map(lambda x:x[1]).collect()
simsit=IxI_sim_dafile.map(lambda x:x[2]).collect()
IxI_sim= sm.csr_matrix((simsit, (it1, it2)))
IxI_sim.setdiag(0)


In [ ]:
k=75
nz_it_row,nz_it_col=IxI_sim.nonzero()
actual_row=-1
actual_top=[]
for i in range(len(nz_it_col)):
    it_row=nz_it_row[i]
    it_col=nz_it_col[i]
    if it_row!=actual_row:
        actual_row=it_row
        actual_top=IxI_sim.getrow(it_row).toarray().argpartition(-k)[-k:]
    if it_col not in actual_top:
        UxU_sim[it_row,it_col]=0
    print(i/len(nz_it_row))

In [ ]:
UxI_pred_CFI=UxI.dot(IxI_sim.T)

In [ ]:
def medRank(user,rank1,rank2,rank3,rank4):
    top=list()
    counterDic = defaultdict(int)
    orderedDic = defaultdict(list)
    already_voted=grouped_rates_dic[user]
    nrRanks=4
    doR1=True
    doR2=True
    doR3=True
    doR4=True

    for i in range(len(rank1)):
        #se non è stato beccato un rating predetto a 0 in questo rank
        
        if doR1:
            #prendi il rating massimo
            item1=rank1.argmax()
            #se il rating massimo non è zero
            if rank1[item1] > 0.0:
                rank1[item1]=-9
                if item1 not in already_voted:
                    count = counterDic[item1]
                    counterDic[item1]+=1
                    if counterDic[item1] >= nrRanks:
                        top+=[item1]
                        orderedDic[count].remove(item1)
                    else:
                        if count > 0:
                            orderedDic[count].remove(item1)
                        orderedDic[count + 1] += [item1]
                if len(top)>=5:
                    break
            #altrimenti smetti di prendere in considerazione questo rank
            else:
                doR1=False
                #nrRanks-=1

        if doR2:
            item2=rank2.argmax()
            if rank2[item2] > 0.0:
                rank2[item2]=-9
                if item2 not in already_voted:
                    count = counterDic[item2]
                    counterDic[item2]+=1
                    if counterDic[item2] >= nrRanks:
                        top+=[item2]
                        orderedDic[count].remove(item2)
                    else:
                        if count > 0:
                            orderedDic[count].remove(item2)
                        orderedDic[count + 1] += [item2]
                if len(top)>=5:
                    break
            else:
                doR2=False
        
        if doR3:
            item3=rank3.argmax()
            if rank3[item3] > 0.0:
                rank3[item3]=-9
                if item3 not in already_voted:
                    count = counterDic[item3]
                    counterDic[item3]+=1
                    if counterDic[item3] >= nrRanks:
                        top+=[item3]
                        orderedDic[count].remove(item3)
                    else:
                        if count > 0:
                            orderedDic[count].remove(item3)
                        orderedDic[count + 1] += [item3]
                if len(top)>=5:
                    break
            else:
                doR3=False
        
        if doR4:
            item4=rank4.argmax()
            if rank4[item4] > 0.0:
                rank4[item4]=-9
                if item4 not in already_voted:
                    count = counterDic[item4]
                    counterDic[item4]+=1
                    if counterDic[item4] >= nrRanks:
                        top+=[item4]
                        orderedDic[count].remove(item4)
                    else:
                        if count > 0:
                            orderedDic[count].remove(item4)
                        orderedDic[count + 1] += [item4]
                if len(top)>=5:
                    break
            else:
                doR4=False
                
                
        
        if (not doR1) and (not doR2) and (not doR3) and (not doR4):
            break


    key_list = list(orderedDic.keys())
    max_rank = None
    if len(key_list) > 0:
        max_rank = max(key_list)
        while len(orderedDic[max_rank]) == 0:
            key_list.remove(max_rank)
            if len(key_list) > 0:
                max_rank = max(key_list)
            else:
                max_rank = None
                break
    if max_rank != None:
        for i in range(5 - len(top)):
            top += [orderedDic[max_rank].pop(0)]
            while len(orderedDic[max_rank]) == 0:
                key_list.remove(max_rank)
                if len(key_list) > 0:
                    max_rank = max(key_list)
                else:
                    max_rank = None
                    break
            if max_rank == None:
                break
    return top

In [ ]:
c=0
f = open('submission_Hyb_medrank_withKNN.csv', 'wt')
writer = csv.writer(f)
writer.writerow(('userId','RecommendedItemIds'))
for user in test_users:

    top5=medRank(user,UxI_pred_CB.getrow(user).toarray()[0],UxI_pred_CBS.getrow(user).toarray()[0],UxI_pred_CFU.getrow(user).toarray()[0],UxI_pred_CFI.getrow(user).toarray()[0])
   
    iterator = 0
    for i in range(5 - len(top5)):
        
        while (item_ratings_mean[iterator] in grouped_rates_dic[user]) or (item_ratings_mean[iterator] in top5):            
            iterator = iterator + 1
        top5 += [item_ratings_mean[iterator]]        
        iterator+=1

    
    
    c+=1
    print(c)
    writer.writerow((user, '{0} {1} {2} {3} {4}'.format(top5[0], top5[1], top5[2], top5[3], top5[4])))

f.close()


In [ ]:
def bordaAggr(rank1,rank2,rank3,rank4):
    nrItems=UxI.shape[1]
    result=[0]*nrItems
    rg=100
    for i in range(rg):
        item1=rank1.argmax()
        item2=rank2.argmax()
        item3=rank3.argmax()
        item4=rank4.argmax()

        if rank1[item1]>0.0:
            result[item1]+=((rg-i)*5.8)
        rank1[item1]=-9

        if rank2[item2]>0.0:
            result[item2]+=((rg-i)*3.6)
        rank2[item2]=-9

        if rank3[item3]>0.0:
            result[item3]+=((rg-i)*2)
        rank3[item3]=-9

        if rank4[item4]>0.0:
            result[item4]+=((rg-i)*1)
        rank4[item4]=-9

    return sm.csr_matrix(result)

In [ ]:
c=0
f = open('submission_borda_100_5.8_3.6_2_1_withknn.csv', 'wt')
writer = csv.writer(f)
writer.writerow(('userId','RecommendedItemIds'))
for user in test_users:
    top=[0,0,0,0,0]

    user_predictions=bordaAggr(UxI_pred_CB.getrow(user).toarray()[0],UxI_pred_CBS.getrow(user).toarray()[0],UxI_pred_CFU.getrow(user).toarray()[0],UxI_pred_CFI.getrow(user).toarray()[0])
    iterator = 0
    for i in range(5):
        prediction = user_predictions.argmax()
        while prediction in grouped_rates_dic[user] and prediction != 0:
            user_predictions[0,prediction]=-9
            prediction=user_predictions.argmax()
        if prediction == 0:
            prediction = item_ratings_mean[iterator]
            while prediction in grouped_rates_dic[user] or prediction in top:
                iterator += 1
                prediction = item_ratings_mean[iterator]
            iterator += 1
        else:
            user_predictions[0,prediction]=-9
        top[i]=prediction
    c+=1
    print(c)
    writer.writerow((user, '{0} {1} {2} {3} {4}'.format(top[0], top[1], top[2], top[3], top[4])))

f.close()

In [ ]:
UxI_pred=UxI_pred_CB.multiply(17)+UxI_pred_CFU.multiply(3)+UxI_pred_CFI.multiply(2)

In [ ]:
c=0
f = open('submission_CF_sum_super_17_3_2_withknn.csv', 'wt')
writer = csv.writer(f)
writer.writerow(('userId','RecommendedItemIds'))
for user in test_users:
    top=[0,0,0,0,0]

    user_predictions=UxI_pred.getrow(user)
    iterator = 0
    for i in range(5):
        prediction = user_predictions.argmax()
        while prediction in grouped_rates_dic[user] and prediction != 0:
            user_predictions[0,prediction]=-9
            prediction=user_predictions.argmax()
        if prediction == 0:
            prediction = item_ratings_mean[iterator]
            while prediction in grouped_rates_dic[user] or prediction in top:
                iterator += 1
                prediction = item_ratings_mean[iterator]
            iterator += 1
        else:
            user_predictions[0,prediction]=-9
        top[i]=prediction
    c+=1
    print(c)
    writer.writerow((user, '{0} {1} {2} {3} {4}'.format(top[0], top[1], top[2], top[3], top[4])))

f.close()